# 項目：分析Netfilx電影演員評分數據

## 分析目標

此數據分析的目的是，整理不同流派影視作品，如喜劇片、動作片或科幻片中，各演員出演作品的平均IMDB評分，從而挖掘各個流派中的高評分作品演員。

## 簡介

原始數據集紀錄了截止至2022年7月美國地區可觀看的所有Netfilx電視劇及電影數據。數據集包含兩個數據表：`titles.csv`和`credits.csv`。  

`titles.csv`包含電影及電視劇相關資訊，包含影視作品ID、標題、類型、描述、流派、IMDB評分等。
`credits.csv`包含超過7萬名出現在Netfilx影視作品的導演及演員資訊，包括名字、影視作品ID、人物名、演職員(導演/演員)類型等。  

`titles.csv`欄位的含義如下：  

- `id`：影視作品ID。
- `title`：影視作品標題。
- `show_type`：作品類型、電視節目或電影。
- `description`：簡短描述。
- `release_year`：發布年份。
- `age_certification`：適齡認證。
- `runtime`：每集電視劇或電影的長度。
- `genres`：流派類型列表。
- `production_countries`：出品國家列表。
- `seasons`：如果是電視劇，則為季數。
- `imdb_id`：IMDB的ID。
- `imdb_score`：IMDB的評分。
- `imdb_votes`：IMDB的投票數。
- `tmdb_popularity`：TMDB的流行度。
- `tmdb_score`：TMDB的評分。

`credits.csv`欄位的含義如下：  

- `person_ID`：演職員ID。
- `id`：參與的影視作品ID。
- `name`：姓名。
- `character_name`：角色姓名。
- `role`：演職員類型，演員或導演。。

## 讀取數據 

導入數據分析所需的函示庫，並透過Pandas的`read_csv`函數，將原始數據文件"titles.csv"里的數據內容，解析為DataFrame並賦值給變量`original_titles`。  
將原始數據文件"credits_csv"里的數據內容，解析為DataFrame並賦值給變量`original_credits`。

In [1]:
import pandas as pd

In [2]:
original_titles = pd.read_csv("titles.csv")
original_credits = pd.read_csv("credits.csv")

In [3]:
original_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [4]:
original_credits.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


## 評估和清洗數據

在這部分，我將對前面建立的`original_titles`及`original_credits`的DataFrame所包含的數據進行評估。評估主要從兩個方面進行結構和內容，及整齊度與乾淨度。  
1. 數據的結構性問題指不符合以下標準
- 每列是一個變量
- 每行是一個觀察值
- 每個單元格是一個值
2. 數據的內容性問題包含以下
- 丟失數據
- 重複數據
- 無效數據  

為了區分經過清洗的數據和原始的數據，創建新的變量`cleaned_titles`，讓他為`original_titles`複製出的副本，以及創建新的變量`cleaned_credits`，讓他為`original_credits`複製出的變量。之後清洗步驟將被運用在`cleaned_titles`與`cleaned_credits`之上。

In [5]:
cleaned_titles = original_titles.copy()
cleaned_credits = original_credits.copy()

### 數據整齊度

In [6]:
cleaned_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


從數據的前部分來看，`cleaned_titles`裡的`genres`和`production_countries`的變量中包含多個值，應當進行拆分。

先提取任一個`genres`變量的值進行觀察。

In [7]:
cleaned_titles["genres"][1]

"['drama', 'crime']"

雖然`genres`表示形式的列表，但其實際型別非字串列表，而是字串，故無法直接使用`value_counts`統計各個值出現的次數。可以使用`eval`函數，可以將字串轉成表達式。

In [8]:
cleaned_titles["genres"] = cleaned_titles["genres"].apply(lambda s: eval(s))
cleaned_titles["genres"][1]

['drama', 'crime']

轉換為列表後，就能用DataFrame的`explode`方法，可以將列表值拆成單獨欄位。

In [9]:
cleaned_titles = cleaned_titles.explode("genres")
cleaned_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300


接下來，針對`production_countries`欄位也是一樣的流程。每個觀察值的`production_countries`值並非單個流派，而是一系列流派。  
先提取任一個`production_countries`變量值進行觀察。

In [10]:
cleaned_titles["production_countries"][1]

1    ['US']
1    ['US']
Name: production_countries, dtype: object

可以得出，`production_countries`也是一樣的問題，雖然表示形式是列表，但實際類型非字串列表，而是字串，需再次使用`eval`函數進行類型轉換，並檢查結果。

In [11]:
cleaned_titles["production_countries"] = cleaned_titles["production_countries"].apply(lambda s: eval(s))
cleaned_titles["production_countries"][0]

['US']

確認類型轉換完畢後，還是用`explode`方法，把欄位值拆分成單獨的行。

In [12]:
cleaned_titles = cleaned_titles.explode("production_countries")
cleaned_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300


在處理完`cleaned_titles`的結構性問題後，查看`cleaned_credits`。

In [13]:
cleaned_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300


從數據的前部分來看，`cleaned_credits`數據符合結構性標準，因此不存在結構性問題。

### 數據乾淨度

接下來透過`info`，對數據內容進行初步暸解。

In [14]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64(2), 

從輸出結果來看，`cleaned_titles`數據夠有17818條觀察值，`title`、`description`、`age_certification`、`genres`、`production_countries`、`seasons`、`imdb_id`、`imdb_score`、`tmdb_popularity`、`tmdb_score`、`imdb_scores`變量均存在缺失值，將在後續進行評估和清洗。  
此外，`release_year`表示年份，數據類型不應為數字，應為日期，故需進行類型轉換。

In [16]:
cleaned_titles["release_year"] = pd.to_datetime(cleaned_titles["release_year"], format="%Y")
cleaned_titles["release_year"]

0      1945-01-01
1      1976-01-01
1      1976-01-01
2      1972-01-01
2      1972-01-01
          ...    
5847   2021-01-01
5848   2021-01-01
5849   2021-01-01
5849   2021-01-01
5849   2021-01-01
Name: release_year, Length: 17818, dtype: datetime64[ns]

In [17]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


從輸出結果來看，`cleaned_credits`數據共有77801條觀察值，其中`character`變量存在觀察值，將在其後進行評估和清洗。  
此外，`person_id`表示演職員ID，數據類型不應為數字，應為字串，故需進行類型轉換。

In [18]:
cleaned_credits["person_id"] = cleaned_credits["person_id"].astype("string")
cleaned_credits["person_id"]

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: string

### 處理缺失數據

## 清洗數據